In [44]:
import pandas as pd
df_municipis = pd.read_excel('/content/idescat_data.xls', sheet_name='municipis')
df_ist = pd.read_excel('/content/idescat_data.xls', sheet_name='IST')
df_densitat = pd.read_excel('/content/idescat_data.xls', sheet_name='densitat')

In [45]:
df_municipis.shape

(947, 4)

In [46]:
df_ist.shape

(211, 2)

In [47]:
df_densitat.shape

(947, 4)

In [48]:
df = df_municipis.merge(df_densitat, how='left', on='Codi')
df['densitat(habitants/km2)'] = round(df['Població'] / df['Superfície(km2)'],2)
df.head()

,Codi,Nom,Codi comarca,Nom comarca,Altitud(m),Superfície(km2),Població,densitat(habitants/km2)
0,250019,Abella de la Conca,25,Pallars Jussà,956,78.27,183,2.34
1,80018,Abrera,11,Baix Llobregat,105,19.94,12538,628.79
2,250024,Àger,23,Noguera,642,160.57,580,3.61
3,250030,Agramunt,38,Urgell,337,79.65,5411,67.93
4,80023,Aguilar de Segarra,7,Bages,480,43.32,282,6.51


In [49]:
df.shape

(947, 8)

In [50]:
df = df.merge(df_ist, how='left', on='Codi')
df.head()

,Codi,Nom,Codi comarca,Nom comarca,Altitud(m),Superfície(km2),Població,densitat(habitants/km2),Índex socioeconòmic territorial
0,250019,Abella de la Conca,25,Pallars Jussà,956,78.27,183,2.34,NaN
1,80018,Abrera,11,Baix Llobregat,105,19.94,12538,628.79,104.1
2,250024,Àger,23,Noguera,642,160.57,580,3.61,NaN
3,250030,Agramunt,38,Urgell,337,79.65,5411,67.93,93.2
4,80023,Aguilar de Segarra,7,Bages,480,43.32,282,6.51,NaN


In [51]:
df.columns

Index(['Codi', 'Nom ', 'Codi comarca', 'Nom comarca', 'Altitud(m)',
       'Superfície(km2)', 'Població', 'densitat(habitants/km2)',
       'Índex socioeconòmic territorial'],
      dtype='object')

In [52]:
df_ist_missing = pd.read_excel('/content/ist_comarques.xls')
df_ist_missing.head(3)

,Codi,Literal,Índex socioeconòmic territorial
0,1,Alt Camp,95.0
1,2,Alt Empordà,84.2
2,3,Alt Penedès,100.0


In [53]:
df.loc[df['Índex socioeconòmic territorial'].isnull(),'IST_idescat'] = 0
df.loc[~df['Índex socioeconòmic territorial'].isnull(),'IST_idescat'] = 1
df.head(3)

,Codi,Nom,Codi comarca,Nom comarca,Altitud(m),Superfície(km2),Població,densitat(habitants/km2),Índex socioeconòmic territorial,IST_idescat
0,250019,Abella de la Conca,25,Pallars Jussà,956,78.27,183,2.34,NaN,0.0
1,80018,Abrera,11,Baix Llobregat,105,19.94,12538,628.79,104.1,1.0
2,250024,Àger,23,Noguera,642,160.57,580,3.61,NaN,0.0


In [54]:
df_missing =  df.loc[df['Índex socioeconòmic territorial'].isnull()] 
df_complete = df.loc[~df['Índex socioeconòmic territorial'].isnull()] 

In [55]:
df_missing = df_missing.merge(df_ist_missing, how='left', left_on='Codi comarca', right_on='Codi', suffixes=('', '_y'))
df_missing.drop(['Índex socioeconòmic territorial','Codi_y', 'Literal'], axis=1, inplace=True)
df_missing.rename({'Índex socioeconòmic territorial_y': 'Índex socioeconòmic territorial'}, axis=1, inplace=True)
df_missing

,Codi,Nom,Codi comarca,Nom comarca,Altitud(m),Superfície(km2),Població,densitat(habitants/km2),IST_idescat,Índex socioeconòmic territorial
0,250019,Abella de la Conca,25,Pallars Jussà,956,78.27,183,2.34,0.0,96.0
1,250024,Àger,23,Noguera,642,160.57,580,3.61,0.0,89.4
2,80023,Aguilar de Segarra,7,Bages,480,43.32,282,6.51,0.0,97.0
3,170010,Agullana,2,Alt Empordà,166,27.73,863,31.12,0.0,84.2
4,80142,Aiguafreda,41,Vallès Oriental,404,7.90,2512,317.97,0.0,101.5
...,...,...,...,...,...,...,...,...,...,...
731,252555,Vinaixa,18,Garrigues,479,37.59,450,11.97,0.0,90.9
732,431779,Vinebre,30,Ribera d'Ebre,34,26.44,425,16.07,0.0,93.2
733,431785,Vinyols i els Arcs,8,Baix Camp,95,10.83,2124,196.12,0.0,94.5
734,83089,Viver i Serrateix,14,Berguedà,729,66.80,172,2.57,0.0,99.9


In [56]:
df = pd.concat([df_missing, df_complete])
df.sort_values(by='Codi comarca', inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,Codi,Nom,Codi comarca,Nom comarca,Altitud(m),Superfície(km2),Població,densitat(habitants/km2),IST_idescat,Índex socioeconòmic territorial
0,430666,"Garidells, els",1,Alt Camp,132,3.07,190,61.89,0.0,95.0
1,431191,Puigpelat,1,Alt Camp,252,9.48,1163,122.68,0.0,95.0
2,430108,Alió,1,Alt Camp,263,7.25,458,63.17,0.0,95.0
3,431135,"Pont d'Armentera, el",1,Alt Camp,349,21.67,488,22.52,0.0,95.0
4,431080,"Pla de Santa Maria, el",1,Alt Camp,381,34.99,2348,67.10,0.0,95.0


In [57]:
df.to_csv('idescat_data_final.csv', index=False)